In [1]:
import pandas as pd
import re
import numpy as np
import dill

from tqdm import tqdm
from collections import defaultdict, Counter

import warnings

warnings.filterwarnings("ignore")

## MIMIC IV

In [2]:
PATIENTS = pd.read_csv('mimiciv/3.1/hosp/PATIENTS.csv.gz')
ADMISSIONS = pd.read_csv('mimiciv/3.1/hosp/ADMISSIONS.csv.gz')
ICUSTAYS = pd.read_csv('mimiciv/3.1/icu/ICUSTAYS.csv.gz')

ADMISSIONS.columns = ADMISSIONS.columns.str.upper()
ICUSTAYS.columns = ICUSTAYS.columns.str.upper()
PATIENTS.columns = PATIENTS.columns.str.upper()

ADMISSIONS = ADMISSIONS.rename(columns={'ETHNICITY':'RACE'})
ICUSTAYS = ICUSTAYS.rename(columns={'STAY_ID':'ICUSTAY_ID'})

In [3]:
def MIMICiv(ADMISSIONS,ICUSTAYS,PATIENTS):
    # Select relevant columns for ADMISSIONS, ICUSTAYS, and PATIENTS
    ADMISSIONS = ADMISSIONS[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE']]
    ICUSTAYS = ICUSTAYS[['HADM_ID', 'ICUSTAY_ID', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'INTIME', 'OUTTIME', 'LOS']]
    PATIENTS['DOB'] = PATIENTS['ANCHOR_YEAR'] - PATIENTS['ANCHOR_AGE']
    PATIENTS = PATIENTS[['SUBJECT_ID', 'GENDER', 'DOB', 'DOD','ANCHOR_YEAR_GROUP']]
    
    # Merge the datasets
    df = pd.merge(PATIENTS, ADMISSIONS, on='SUBJECT_ID', how='left')
    df = pd.merge(df, ICUSTAYS, on='HADM_ID', how='left')
    print(df.shape,len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()),'\n',df.ANCHOR_YEAR_GROUP.value_counts())
    
    # Convert date columns to datetime
    date_columns = ['ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME','DOD']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)

    # Drop rows with missing critical columns
    df = df.dropna(subset=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME', 'OUTTIME', 'DOB'])
    df = df.sort_values(by=['SUBJECT_ID', 'ADMITTIME'])
    print('dropna', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))

    # Sort the dataframe by 'SUBJECT_ID' and 'INTIME'
    df = df.sort_values(by=['SUBJECT_ID', 'INTIME'])
    
    # Create 'DOB' as a complete date by assuming January 1st for each year
    df['DOB'] = pd.to_datetime(df['DOB'].astype(str) + '-01-01')
    
    # Calculate 'AGE' based on 'ADMITTIME' and 'DOB', and adjust for ages above 89
    df['AGE'] = ((df['ADMITTIME'].dt.date - df['DOB'].dt.date) / 365.242).dt.days
    df['AGE'] = df['AGE'].apply(lambda x: 91 if x > 89 else x)
    
    # Filter out patients under 18
    df = df[df['AGE'] >= 18]
    print('AGE >= 18',df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))

    # Keep rows with LOS greater than 1
    df = df[df['LOS'] >= 1]
    print('LOS greater than 1', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))
    
    # Select specific columns for the final dataframe
    df = df[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME', 'DISCHTIME', 'OUTTIME', 
             'GENDER', 'DOB', 'DOD', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE', 
             'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'LOS', 'AGE', 'ANCHOR_YEAR_GROUP']]

    # Create a flag for the first admission for each patient
    df['FIRST_HADM'] = df.groupby('SUBJECT_ID')['ADMITTIME'].transform(lambda x: x == x.min()).astype(int)
    # Create a flag for the first ICU stay for each patient-HADM combination
    df['FIRST_ICU'] = df.groupby(['SUBJECT_ID', 'HADM_ID'])['INTIME'].transform(lambda x: x == x.min()).astype(int)
    
    df = df[df['FIRST_ICU'] == 1]
    print('All_HADM, FIRST_ICU', df.shape, len(df.SUBJECT_ID.unique()),len(df.HADM_ID.unique()),len(df.ICUSTAY_ID.unique()))

    df['DIEINHOSPITAL'] = ((df['ADMITTIME'] <= df['DOD']) & (df['DOD'] <= df['DISCHTIME'])).astype(int)
    df['DIEINICU'] = ((df['INTIME'] <= df['DOD']) & (df['DOD'] <= df['OUTTIME'])).astype(int)

    df['NEXT_ADMIT'] = df.groupby('SUBJECT_ID')['ADMITTIME'].shift(-1)
    df['DAYS_TO_READMIT'] = (df['NEXT_ADMIT'] - df['ADMITTIME']).dt.days
    df['Readmission_30'] = df['DAYS_TO_READMIT'].apply(lambda x: 1 if pd.notnull(x) and x <= 30 else 0)
    df['Readmission_60'] = df['DAYS_TO_READMIT'].apply(lambda x: 1 if pd.notnull(x) and x <= 60 else 0)

    df['HOURS_FROM_ADMIT'] = (df['INTIME'] - df['ADMITTIME']).dt.total_seconds() / 3600
    df['ICU_within_12hr_of_admit'] = df['HOURS_FROM_ADMIT'].apply(lambda x: 1 if 0 <= x <= 12 else 0)

    ICUSTAYS = ICUSTAYS[ICUSTAYS.HADM_ID.isin(df.HADM_ID)]
    ICUSTAYS['Multiple_ICUs'] = (ICUSTAYS.groupby('HADM_ID')['ICUSTAY_ID']
                    .transform('count')
                    .gt(1) 
                    .astype(int))  
    ICUSTAYS = ICUSTAYS[['HADM_ID','Multiple_ICUs']].drop_duplicates(keep='first')
    df = pd.merge(df,ICUSTAYS,on='HADM_ID',how='left')
    

    df.drop(columns=['NEXT_ADMIT', 'DAYS_TO_READMIT'], inplace=True)

    df['SUBJECT_ID'] = df['SUBJECT_ID'].astype(int)
    df['HADM_ID'] = df['HADM_ID'].astype(int)
    df['ICUSTAY_ID'] = df['ICUSTAY_ID'].astype(int)

    print('------------------------------------------------------------------------')
    print(df.shape, 'SUBJECT_ID:',len(df.SUBJECT_ID.unique()),'HADM_ID:',len(df.HADM_ID.unique()), 'ICUSTAY_ID:',len(df.ICUSTAY_ID.unique()))
    print('DIEINHOSPITAL:',Counter(df['DIEINHOSPITAL']))
    print('DIEINICU:',Counter(df['DIEINICU']))
    print('Readmission_60:',Counter(df['Readmission_60']))
    print('ICU_within_12hr_of_admit:',Counter(df['ICU_within_12hr_of_admit']))
    print('Multiple_ICUs:',Counter(df['Multiple_ICUs']))

    return df

In [4]:
All_df = MIMICiv(ADMISSIONS,ICUSTAYS,PATIENTS)

(696419, 18) 364627 546029 94459 
 2008 - 2010    265223
2011 - 2013    145899
2014 - 2016    121728
2017 - 2019     99377
2020 - 2022     64192
Name: ANCHOR_YEAR_GROUP, dtype: int64
dropna (94444, 18) 65355 85229 94444
AGE >= 18 (94444, 19) 65355 85229 94444
LOS greater than 1 (74829, 19) 54551 68546 74829
All_HADM, FIRST_ICU (68546, 21) 54551 68546 68546
------------------------------------------------------------------------
(68546, 28) SUBJECT_ID: 54551 HADM_ID: 68546 ICUSTAY_ID: 68546
DIEINHOSPITAL: Counter({0: 60729, 1: 7817})
DIEINICU: Counter({0: 63391, 1: 5155})
Readmission_60: Counter({0: 64117, 1: 4429})
ICU_within_12hr_of_admit: Counter({1: 47150, 0: 21396})
Multiple_ICUs: Counter({0: 61177, 1: 7369})


In [5]:
All_df.columns

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME',
       'DISCHTIME', 'OUTTIME', 'GENDER', 'DOB', 'DOD', 'DEATHTIME',
       'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'RACE', 'FIRST_CAREUNIT',
       'LAST_CAREUNIT', 'LOS', 'AGE', 'ANCHOR_YEAR_GROUP', 'FIRST_HADM',
       'FIRST_ICU', 'DIEINHOSPITAL', 'DIEINICU', 'Readmission_30',
       'Readmission_60', 'HOURS_FROM_ADMIT', 'ICU_within_12hr_of_admit',
       'Multiple_ICUs'],
      dtype='object')

## Key Variable

In [ ]:
keep_icu = list(set(chart.ICUSTAY_ID.unique())&set(lab.ICUSTAY_ID.unique())&set(proc.ICUSTAY_ID.unique())&set(Meds.ICUSTAY_ID.unique()))
print(len(keep_icu))
df = All_df[All_df.ICUSTAY_ID.isin(keep_icu)]

## Bacterial culture during ICU

In [ ]:
microbiologyevents = pd.read_csv('mimiciv/3.1/hosp/microbiologyevents.csv.gz')
microbiologyevents.columns = microbiologyevents.columns.str.upper()

microevents = microbiologyevents[microbiologyevents['HADM_ID'].isin(df['HADM_ID'])]
print(microbiologyevents.shape,microevents.shape)
del(microbiologyevents)

In [ ]:
merged = pd.merge(microevents, df[['HADM_ID', 'ICUSTAY_ID', 'ADMITTIME', 'INTIME','DISCHTIME', 'OUTTIME']], on =['HADM_ID'], how='left')
merged = merged.dropna(subset=['ICUSTAY_ID'])

datetime_cols = ['ADMITTIME', 'INTIME','DISCHTIME', 'OUTTIME', 'CHARTTIME']
merged[datetime_cols] = merged[datetime_cols].apply(pd.to_datetime)

In [ ]:
SSC_icu_window_criteria = merged['CHARTTIME'].between(
    merged['INTIME']- pd.DateOffset(1),
    merged['OUTTIME'])
merged = merged[SSC_icu_window_criteria]
merged.shape

In [ ]:
merged = merged.sort_values(['ICUSTAY_ID','INTIME','CHARTTIME','ORG_NAME']).drop_duplicates(['ICUSTAY_ID','CHARTTIME', 'TEST_NAME','ORG_NAME','AB_NAME', 'INTERPRETATION']) 
print('SUBJECT_ID:',len(merged.SUBJECT_ID.unique()),'HADM_ID:',len(merged.HADM_ID.unique()), 'ICUSTAY_ID:',len(merged.ICUSTAY_ID.unique()))

In [ ]:
df = df[df.ICUSTAY_ID.isin(merged.ICUSTAY_ID)]